#### ViT Model for Classification of All Defects (including nones)
ViT Model from Google [paper](https://arxiv.org/abs/2010.11929)
using Keras implementation from faustomorales [GitHub](https://github.com/faustomorales/vit-keras)

Data preprocessing:
- Resized to 224x224 with no filters
- None is randomly undersampled to 30,000.

ViT model:
- Size = B16
- Patch size = 16
- Using included top

In [ ]:
#### RUN VIT REQUIREMENTS NOTEBOOK FIRST ####
# need to install required packages to environment before running the model

In [ ]:
# import libraries
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras import datasets, layers, models, losses, optimizers, regularizers, callbacks
from vit_keras import vit, utils

import os
import time
import numpy as np
import pandas as pd
import cv2
from skimage.morphology import skeletonize, thin

import helpers as helper
from keras_model_s3_wrapper import *

import boto3
import pickle5 as pickle
s3 = boto3.resource('s3')
bucket_name = 'wafer-capstone'
my_bucket = s3.Bucket(bucket_name)

In [ ]:
tf.__version__

In [ ]:
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.compat.v1.Session(config=config)

In [ ]:
tf.config.list_physical_devices(device_type=None)

In [ ]:
# specify variables
path = 'processed_data/vit'
filename = 'WM-clean-vit224-undersampled'

result_path = 'results_vit'
model_id = 'vit-b16'
data_id = '224-undersampled'
note = '' # -optional

In [ ]:
!free -h

In [ ]:
# load dataset
start = time.time()

data_key = f'{path}/{filename}.npz'
data_obj = my_bucket.Object(train_key).get()['Body'].read()
data = np.load(data_obj, allow_pickle=True)

print('Wall time: {:.3f} seconds'.format(time.time() - start))
data.files

In [ ]:
!free -h

#### Data set-up

In [ ]:
# prepare inputs
start = time.time()

x_train = data['wafermap'][data['dataset']=='train']
x_val = data['wafermap'][data['dataset']=='dev']
x_test = data['wafermap'][data['dataset']=='test']

print('Wall time: {:.3f} seconds'.format(time.time() - start))
print(f'Train: {x_train.shape}')
print(f'Dev: {x_val.shape}')
print(f'Test: {x_test.shape}')

In [ ]:
# prepare labels for supervised learning
# note: make sure labels are integers if using sparse categorical cross entropy
start = time.time()

y_train = data['labels'][data['dataset']=='train']
y_train = data['labels'][data['dataset']=='dev']
y_train = data['labels'][data['dataset']=='test']

print("Wall time: {:.2f} seconds".format(time.time() - start))
# sanity check
# expected: type = int, min = 0, max = 8
print(type(y_train[0]))
print(min(y_train), min(y_val), min(y_test))
print(max(y_train), max(y_val), max(y_test))

In [ ]:
!free -h

#### Model

In [ ]:
# fine-tune using pre-trained vit model
image_size = 224
model = vit.vit_b16(
    image_size=image_size,
    activation='sigmoid',
    pretrained=True,
    include_top=True,
    pretrained_top=False,
    classes=9
)

model.summary()

In [ ]:
# set model optimizer and metrics
opt = optimizers.Adam(learning_rate=0.0001)
model.compile(optimizer='adam', loss=losses.sparse_categorical_crossentropy, metrics=['accuracy'])

In [ ]:
!free -h

In [ ]:
# run model
start = time.time()

history = model.fit(x_train, y_train, validation_data=(x_val, y_val), batch_size=64, epochs=30)

print("Wall time: {:.2f} seconds".format(time.time() - start))

In [ ]:
# visualize accuracy and loss history
fig, axs = plt.subplots(2, 1, figsize=(15,15))

axs[0].plot(history.history['loss'])
axs[0].plot(history.history['val_loss'])
axs[0].title.set_text('Training Loss vs Validation Loss')
axs[0].legend(['Train', 'Val'])

axs[1].plot(history.history['accuracy'])
axs[1].plot(history.history['val_accuracy'])
axs[1].title.set_text('Training Accuracy vs Validation Accuracy')
axs[1].legend(['Train', 'Val'])

#### Model results

In [ ]:
# save model to S3
s3_save_keras_model(model, f'{model_id}-{data_id}{note}')

In [ ]:
# compute model results on test set
start = time.time()
results = model.evaluate(x_test, y_test)
print("Wall time: {:.2f} seconds".format(time.time() - start))
print(results)

In [ ]:
# generate predictions for model analysis
start = time.time()
y_pred = model.predict(x_test)
y_max = np.argmax(y_pred, axis=1).astype(np.uint8)
predictions = [y_max, y_pred]
print("Wall time: {:.2f} seconds".format(time.time() - start))

In [ ]:
# save predictions to local instance
with open(f'{result_path}/{model_id}-{data_id}{note}.pkl', "wb") as f:
    pickle.dump(predictions, f)

In [ ]:
# plot confusion matrix
helper.plot_confusion_matrix(y_test, y_max, mode='all', normalize=True)

In [ ]:
# plot confusion matrix counts
helper.plot_confusion_matrix(y_test, y_max, mode='all', normalize=False)